# Logistic Regression

In [ ]:
import joblib
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Load shared label encoder
label_encoder = joblib.load("label_encoder.pkl")
valid_bowler_types = list(label_encoder.classes_)

# Prompt for number of deliveries
while True:
    try:
        seq_len = int(input("How many deliveries have been bowled? (1 to 5): "))
        if 1 <= seq_len <= 5:
            break
        print("❌ Please enter a number between 1 and 5.")
    except ValueError:
        print("❌ Invalid input. Please enter an integer.")

# Collect delivery inputs
def get_validated_delivery_input(index):
    print(f"\nEnter details for delivery #{index + 1}:")

    def get_float(prompt):
        while True:
            try:
                return float(input(prompt))
            except ValueError:
                print("  ❌ Invalid number. Try again.")

    landing_x = get_float("  landing_x: ")
    landing_y = get_float("  landing_y: ")
    ended_x = get_float("  ended_x: ")
    ended_y = get_float("  ended_y: ")
    ball_speed = get_float("  ball_speed (km/h): ")
    ovr = get_float("  over (e.g., 2.3 for 3rd ball of 3rd over): ")

    while True:
        bowler_type = input(f"  bowler_type {valid_bowler_types}: ")
        if bowler_type in valid_bowler_types:
            break
        print("  ❌ Invalid bowler_type. Choose from the list above.")

    encoded_type = label_encoder.transform([bowler_type])[0]
    return [landing_x, landing_y, ended_x, ended_y, ball_speed, encoded_type, ovr]

# Gather inputs
manual_inputs = [get_validated_delivery_input(i) for i in range(seq_len)]
sequence = np.array(manual_inputs, dtype=np.float32)

# Load appropriate scaler and apply it
try:
    scaler = joblib.load(f"scaler_seq{seq_len}.pkl")
except FileNotFoundError:
    print(f"❌ No trained scaler found for seq_len={seq_len}. Please run training first.")
    exit()

numerical_indices = [0, 1, 2, 3, 4, 6]
for i in numerical_indices:
    sequence[:, i] = scaler.transform(sequence[:, i].reshape(-1, 1)).reshape(-1)

# Flatten and predict
input_vector = sequence.flatten().reshape(1, -1)
try:
    model = joblib.load(f"logistic_regression_seq{seq_len}.pkl")
except FileNotFoundError:
    print(f"❌ No trained model found for seq_len={seq_len}. Please run training first.")
    exit()

prediction = model.predict(input_vector)[0]
probability = model.predict_proba(input_vector)[0][1]

# Show result
print("\n🧠 Prediction Result:")
print("  ✅ Predicted class for the Next delivery: ", "Deceptive" if prediction == 1 else "Not Deceptive")
print(f"  🔍 Confidence of Result: {probability * 100:.2f}%")


How many deliveries have been bowled? (1 to 5): 1

Enter details for delivery #1:
  landing_x: 1.2
  landing_y: 2.3
  ended_x: 1.4
  ended_y: 1.4
  ball_speed (km/h): 123
  over (e.g., 2.3 for 3rd ball of 3rd over): Fast
  ❌ Invalid number. Try again.
  over (e.g., 2.3 for 3rd ball of 3rd over): 2.1
  bowler_type ['Fast', 'Medium', 'Spin', 'Unknown']: Fast

🧠 Prediction Result:
  ✅ Predicted class for the LAST delivery:  Not Deceptive
  🔍 Confidence of being deceptive: 4.83%


# LSTM Model

In [2]:
import numpy as np
import joblib
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder

# Load label encoder
label_encoder = joblib.load("label_encoder.pkl")
valid_bowler_types = list(label_encoder.classes_)

# Get user-defined sequence length
while True:
    try:
        seq_len = int(input("How many deliveries have been bowled? (1 to 5): "))
        if 1 <= seq_len <= 5:
            break
        print("❌ Please enter a number between 1 and 5.")
    except ValueError:
        print("❌ Invalid input. Please enter an integer.")

# Get input features for each delivery
def get_validated_delivery_input(index):
    print(f"\nEnter details for delivery #{index + 1}:")

    def get_float(prompt):
        while True:
            try:
                return float(input(prompt))
            except ValueError:
                print("  ❌ Invalid number. Try again.")

    landing_x = get_float("  landing_x: ")
    landing_y = get_float("  landing_y: ")
    ended_x = get_float("  ended_x: ")
    ended_y = get_float("  ended_y: ")
    ball_speed = get_float("  ball_speed (km/h): ")
    ovr = get_float("  over (e.g., 2.3 for 3rd ball of 3rd over): ")

    while True:
        bowler_type = input(f"  bowler_type {valid_bowler_types}: ")
        if bowler_type in valid_bowler_types:
            break
        print("  ❌ Invalid bowler_type. Choose from the list above.")

    encoded_type = label_encoder.transform([bowler_type])[0]
    return [landing_x, landing_y, ended_x, ended_y, ball_speed, encoded_type, ovr]

manual_inputs = [get_validated_delivery_input(i) for i in range(seq_len)]
sequence = np.array(manual_inputs, dtype=np.float32)

# Scale numeric inputs
try:
    scaler = joblib.load(f"scaler_lstm_seq{seq_len}.pkl")
except FileNotFoundError:
    print(f"❌ Scaler not found for seq_len={seq_len}. Run training first.")
    exit()

for i in [0, 1, 2, 3, 4, 6]:
    sequence[:, i] = scaler.transform(sequence[:, i].reshape(-1, 1)).reshape(-1)

# Reshape to 3D for LSTM: (1, seq_len, features)
input_vector = sequence.reshape(1, seq_len, -1)

# Load and predict using the LSTM model
try:
    model = load_model(f"lstm_model_seq{seq_len}.h5")
except FileNotFoundError:
    print(f"❌ LSTM model not found for seq_len={seq_len}. Run training first.")
    exit()

probability = model.predict(input_vector)[0][0]
prediction = int(probability > 0.5)

# Output
print("\n🧠 LSTM Prediction Result:")
print("  ✅ Predicted class for the Next delivery: ", "Deceptive" if prediction == 1 else "Not Deceptive")
print(f"  🔍 Confidence of Result: {probability * 100:.2f}%")


How many deliveries have been bowled? (1 to 5): 1

Enter details for delivery #1:
  landing_x: 1.2
  landing_y: 1.3
  ended_x: -1.4
  ended_y: -1.5
  ball_speed (km/h): 134
  over (e.g., 2.3 for 3rd ball of 3rd over): 2.1
  bowler_type ['Fast', 'Medium', 'Spin', 'Unknown']: Medium


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step

🧠 LSTM Prediction Result:
  ✅ Predicted class for the LAST delivery:  Not Deceptive
  🔍 Confidence of Result: 28.93%
